## These have to be setup before
- https://github.com/tensorflow/models has been cloned to Tensorflow/models
- LabelImg has been cloned
- protoc has been installed and is in Tensorflow/protoc
- Packages have been installed (see requirements.txt)
- Your footage is in the footage folder in video format

# Path Setup

In [2]:
import os;

FOOTAGE_PATH = "Footage/"
LABELIMG_PATH = os.path.join('Tensorflow', 'labelimg')

TRAIN_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'train')
TEST_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'test')

CUSTOM_MODEL_NAME = "HockeyPlayerDetectionModel"
PRETRAINED_MODEL_NAME = "ssd_resnet50_v1_fpn_640x640_coco17_tpu-8"
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join( 'Tensorflow', 'workspace','annotations'),
    'UNLABELED_IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images', 'unlabeled'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
}

files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

for filename in paths.values():
    if not os.path.exists(filename):
        os.makedirs(filename)

def makeabsolute(relative_path):
    return os.path.join("D:\Python\HockeyPlayerDetection", relative_path)

# Image Collection

In [35]:
# Collect given amount of images by going through videos in randomized increments

import random
import cv2

image_amount_target = 100

footage_filenames = os.listdir(FOOTAGE_PATH)

image_amount_pervideo = image_amount_target / len(footage_filenames)

print("Images per video: " + str(image_amount_pervideo))

images_collected = 0

def random_frame_amount(increment_amount):
    return random.randrange(int(increment_amount * 0.9), int(increment_amount * 1.1))

for video_path in footage_filenames:
    video_path_full = os.path.join(FOOTAGE_PATH, video_path)

    vidcap = cv2.VideoCapture(video_path_full)

    vid_len = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

    increment_amount = vid_len / image_amount_pervideo

    frame_index = random_frame_amount(increment_amount)
    while frame_index < vid_len:
        # Set video to specific frame and read it
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        success, image = vidcap.read()

        if success:
            # Get video name without .mp4
            vid_name_noextension = os.path.splitext(video_path)[0]

            # Replace spaces with underscores
            vid_name_noextension.replace(' ', '_')

            full_path_noex = os.path.join(paths['UNLABELED_IMAGE_PATH'], vid_name_noextension)

            # Save image
            cv2.imwrite(full_path_noex + "_" + str(frame_index) + '.jpg', image) 
            
            images_collected += 1

        frame_index += random_frame_amount(increment_amount)

all_files = os.listdir(paths['UNLABELED_IMAGE_PATH'])

# Remove spaces
for filename in all_files:
    os.rename(os.path.join(paths['UNLABELED_IMAGE_PATH'], filename), os.path.join(paths['UNLABELED_IMAGE_PATH'], filename.replace(' ', '_')))

print("Images collected: " + str(images_collected))

Images per video: 5.0
Images collected: 87


#### Filter images

- press d to delete
- s for skip
- q to quit

In [36]:
filenames = os.listdir(paths['UNLABELED_IMAGE_PATH'])

for name in filenames:
    img_path = os.path.join(paths['UNLABELED_IMAGE_PATH'], name)
    image = cv2.imread(img_path)
    
    cv2.imshow(name, image)

    cv2.moveWindow(name, 0, 0)

    k = cv2.waitKey(0)

    cv2.destroyWindow(name)

    if k == ord('d'):
        os.remove(img_path)
        print(img_path + " deleted!")
        continue
    elif k == ord('q'):
        break
    else:
        continue

cv2.destroyAllWindows()

print(str(len(os.listdir(paths['UNLABELED_IMAGE_PATH']))) + " images left!")

Tensorflow\workspace\images\unlabeled\NHL_Jan.08_2022_Chicago_Blackhawks_-_Vegas_Golden_Knight_181376.jpg deleted!
Tensorflow\workspace\images\unlabeled\NHL__Jan.06_2022___San_Jose_Sharks_-_Buffalo_Sabres_167761.jpg deleted!
Tensorflow\workspace\images\unlabeled\NHL__Jan.06_2022___San_Jose_Sharks_-_Buffalo_Sabres_82719.jpg deleted!
Tensorflow\workspace\images\unlabeled\NHL__Jan.06_2022___Winnipeg_Jets_-_Colorado_Avalanche_38570.jpg deleted!
Tensorflow\workspace\images\unlabeled\NHL__Jan.06_2022___Winnipeg_Jets_-_Colorado_Avalanche_79490.jpg deleted!
Tensorflow\workspace\images\unlabeled\NHL__Jan.08_2022___Pittsburgh_Penguins_-_Dallas_Stars_41655.jpg deleted!
Tensorflow\workspace\images\unlabeled\NHL__Jan.10_2022__Seattle_Kraken_-_Colorado_Avalanche_121241.jpg deleted!
Tensorflow\workspace\images\unlabeled\NHL__Jan.10_2022__Seattle_Kraken_-_Colorado_Avalanche_161150.jpg deleted!
Tensorflow\workspace\images\unlabeled\NHL__Jan.10_2022__Seattle_Kraken_-_Colorado_Avalanche_81969.jpg deleted

#### Delete collected images CAREFUL

In [16]:
# Delete old collected images
delete_old_files = False

if delete_old_files:
    old_imgs = os.listdir(paths['UNLABELED_IMAGE_PATH'])

    for img_name in old_imgs:
        os.remove(os.path.join(paths["UNLABELED_IMAGE_PATH"], img_name))

# Label Images

In [4]:
# Open LabelImg
!cd {LABELIMG_PATH} && python labelImg.py

# Save labeled images to Tensorflow\workspace\images

Cancel creation.
Image:D:\Python\HockeyPlayerDetection\Tensorflow\workspace\images\unlabeled\NHL___Oct.17_2021__Dallas_Stars_-_Ottawa_Senators_230273.jpg -> Annotation:D:/Python/HockeyPlayerDetection/Tensorflow/workspace/images/unlabeled\NHL___Oct.17_2021__Dallas_Stars_-_Ottawa_Senators_230273.xml
Image:D:\Python\HockeyPlayerDetection\Tensorflow\workspace\images\unlabeled\NHL___Oct.18_2021___Anaheim_Ducks_-_Calgary_Flames_26958.jpg -> Annotation:D:/Python/HockeyPlayerDetection/Tensorflow/workspace/images/unlabeled\NHL___Oct.18_2021___Anaheim_Ducks_-_Calgary_Flames_26958.xml
Image:D:\Python\HockeyPlayerDetection\Tensorflow\workspace\images\unlabeled\NHL___Oct.17_2021__Dallas_Stars_-_Ottawa_Senators_297278.jpg -> Annotation:D:/Python/HockeyPlayerDetection/Tensorflow/workspace/images/unlabeled\NHL___Oct.17_2021__Dallas_Stars_-_Ottawa_Senators_297278.xml
Image:D:\Python\HockeyPlayerDetection\Tensorflow\workspace\images\unlabeled\NHL___Oct.18_2021___Anaheim_Ducks_-_Calgary_Flames_26958.jpg 

### Copy labeled images to train and test folders

In [5]:
import shutil

test_image_amount = 4

all_files = os.listdir(paths['UNLABELED_IMAGE_PATH'])

print(len(all_files))

unlabeled_imgs = []
for filename in all_files:
    # Check if image has been labeled (an xml for the same)
    if filename.endswith('.jpg') and filename.replace('.jpg', '.xml') not in all_files:
        unlabeled_imgs.append(filename)

for filename in unlabeled_imgs:
    if filename in all_files:
        all_files.remove(filename)

print(len(all_files))

test_image_amount *= 2
# Use first images for evaluating
test_filenames = all_files[0:test_image_amount]
# Use rest for training
train_filenames = all_files[test_image_amount:len(all_files)]

# Delete old labeled images
for filename in os.listdir(TEST_PATH):
    os.remove(os.path.join(TEST_PATH, filename))
for filename in os.listdir(TRAIN_PATH):
    os.remove(os.path.join(TRAIN_PATH, filename))

def copylabeledimages(filenames, output_path):
    for filename in filenames:
        shutil.copy(os.path.join(paths['UNLABELED_IMAGE_PATH'], filename), os.path.join(output_path, filename))

# Copy labeled images to their respective directories
copylabeledimages(train_filenames, TRAIN_PATH)
copylabeledimages(test_filenames, TEST_PATH)

print(str((len(all_files) - test_image_amount) / 2) + " images for training")
print(str(test_image_amount / 2) + " images for testing")

194
142
67.0 images for training
4.0 images for testing


# Create Label Map

In [6]:
labels = [{'name':'hockeyplayer', 'id':1}, {'name':'goalie', 'id':2}, {'name':'referee', 'id':3}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

### Verify object detection install

In [3]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

^C


# Generate TF Records

In [ ]:
# Get generation script
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

In [7]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

Successfully created the TFRecord file: Tensorflow\workspace\annotations\train.record
Successfully created the TFRecord file: Tensorflow\workspace\annotations\test.record


# Copy model configuration file

In [15]:
!copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {paths['CHECKPOINT_PATH']}

        1 file(s) copied.


# Update model config file for transfer learning

In [8]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [9]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [10]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(makeabsolute(files['PIPELINE_CONFIG']), "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [11]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 6
pipeline_config.train_config.fine_tune_checkpoint = makeabsolute(os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0'))
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= makeabsolute(files['LABELMAP'])
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [makeabsolute(os.path.join(paths['ANNOTATION_PATH'], 'train.record'))]
pipeline_config.eval_input_reader[0].label_map_path = makeabsolute(files['LABELMAP'])
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [makeabsolute(os.path.join(paths['ANNOTATION_PATH'], 'test.record'))]

In [12]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(makeabsolute(files['PIPELINE_CONFIG']), "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# Training

In [3]:
# Allow dynamic memory allocation
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [4]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps={}".format(makeabsolute(TRAINING_SCRIPT), makeabsolute(paths['CHECKPOINT_PATH']), makeabsolute(files['PIPELINE_CONFIG']), str(2000))

print(command)


python D:\Python\HockeyPlayerDetection\Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=D:\Python\HockeyPlayerDetection\Tensorflow\workspace\models\HockeyPlayerDetectionModel --pipeline_config_path=D:\Python\HockeyPlayerDetection\Tensorflow\workspace\models\HockeyPlayerDetectionModel\pipeline.config --num_train_steps=2000


# Evaluating

In [7]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(makeabsolute(TRAINING_SCRIPT), makeabsolute(paths['CHECKPOINT_PATH']), makeabsolute(files['PIPELINE_CONFIG']), makeabsolute(paths['CHECKPOINT_PATH']))
print(command)

python D:\Python\HockeyPlayerDetection\Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=D:\Python\HockeyPlayerDetection\Tensorflow\workspace\models\HockeyPlayerDetectionModel --pipeline_config_path=D:\Python\HockeyPlayerDetection\Tensorflow\workspace\models\HockeyPlayerDetectionModel\pipeline.config --checkpoint_dir=D:\Python\HockeyPlayerDetection\Tensorflow\workspace\models\HockeyPlayerDetectionModel


In [18]:
!tensorboard --logdir={os.path.join(paths['CHECKPOINT_PATH'], 'eval')}

^C


# Export/Freeze model

In [3]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')

command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])

print(command)

python Tensorflow\models\research\object_detection\exporter_main_v2.py  --input_type=image_tensor --pipeline_config_path=Tensorflow\workspace\models\HockeyPlayerDetectionModel\pipeline.config --trained_checkpoint_dir=Tensorflow\workspace\models\HockeyPlayerDetectionModel --output_directory=Tensorflow\workspace\models\HockeyPlayerDetectionModel\export


# Testing exported model

In [3]:
import tensorflow as tf
import cv2
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import numpy as np

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(os.path.join(paths['OUTPUT_PATH'], 'saved_model'))

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# Labels load
PATH_TO_LABELS = files['LABELMAP']

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

cap = cv2.VideoCapture(R'D:\Python\HockeyPlayerDetection\Footage\Eval\NHL   Oct.13_2022    New York Rangers - Minnesota Wild.mp4')

cap.set(cv2.CAP_PROP_POS_FRAMES, int(cap.get(cv2.CAP_PROP_FRAME_COUNT) / 2))

while cap.isOpened():
    ret, frame = cap.read()

    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    image_np = np.array(frame)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)

    cv2.imshow("Exported model test", cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))

    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

Loading model...Done! Took 7.103670120239258 seconds
